In [ ]:
import uproot
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style('white')

In [ ]:
rcParams.update({'figure.autolayout':True})
rcParams.update({'figure.figsize':[15,8]})

In [ ]:
# file='Example_Bi212_pos648.5_0_-1022.6_seed1.nEXOevents.root'
# suffix=":Event/Sim/SimEvent/SimEvent"
root='/home/thakur/slac_data/PX1M/'

In [ ]:
file='PX1M_Th228_pos648.5_0_-1022.6.root'+':tree'

In [ ]:
file1=root+file
file1

In [ ]:
f=uproot.open(file1)

In [ ]:
f.values()

In [ ]:
f.keys()

In [ ]:
f.show()

In [ ]:
df=f.arrays(f.keys(),library='pd')

In [ ]:
df

In [ ]:
df.columns.values

In [ ]:
apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'
app='(m_nOPCal< (1.064*m_nQ+703))'

In [ ]:
df_filtered=df.query(apply_filter).reset_index(drop=True)

In [ ]:
df_filtered

In [ ]:
ene, weight = df_filtered.energy,df_filtered.weight
component='energy'
isotope='Th228'
location='PX'
plt.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=component)
plt.yscale("log")
plt.ylabel("Counts")
plt.xlabel("Energy [keV]")
plt.title(f"{isotope} [{location}] Energy PDF for SS events in the inner 2000 kg")
plt.legend()
plt.show()

In [ ]:
# fil='sel_filter = (passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0)& (m_nOPCal < 1.064 * m_nQ + 703) & (m_nOPC
# al > 0.644 * m_nQ - 2411) & (~NESTBugFound)
# & (m_DNNvalue>0.85) & (standoff > 100))'

In [ ]:
en=df.energy.values
en

In [ ]:
# plt.hist(en,density=1,bins=500)
# plt.show()

In [ ]:
df.query('3000>energy>2000')['energy']

In [ ]:
energy=f.arrays(["energy"],library='np')

In [ ]:
energy

In [ ]:
fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(projection='3d')
ax.scatter(df['fGenX'],df['fGenY'],df['fGenZ'],color='r')
ax.set_xlabel('fGenX');ax.set_ylabel('fGenY');ax.set_zlabel('fGenZ')
plt.show()

In [ ]:
df['fGenParticleID'].value_counts()#.dropna().value_counts()

In [ ]:
df['fGenParticleID'].to_list()

In [ ]:
df['fGenParticleID'].unique

In [ ]:
from collections import Counter
from  itertools import chain
df_genparticle=pd.Series(Counter(chain(df.fGenParticleID)))
dfgen=df_genparticle.to_frame()
#dfgen=dfgen.rename(columns=['count'])
#df_genparticle.rename(columns=['iso','count'])
dfgen.columns=['num'];#dfgen.index='iso'

In [ ]:
dfgen.plot(kind='bar')

In [ ]:
selection='fXpos'
x=df[selection]
for y in x:
    print(f"length of y[0] {len(y[0])}")
    print(f"y[0] values {y[0]}")


In [ ]:
selection='fYpos'
y=df[selection]
y

In [ ]:
selection='fZpos'
z=df[selection]
z